<a href="https://colab.research.google.com/github/rsaran-BioAI/AGILE/blob/main/Fashion_MNIST_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Download and load the training data
# trainset = datasets.FashionMNIST('/content/drive/MyDrive/F_MNIST_data', download=True, train=True, transform=ToTensor())

NameError: ignored

In [ ]:
# Download and load the test data
# testset = datasets.FashionMNIST('/content/drive/MyDrive/F_MNIST_data/', download=True, train=False, transform=ToTensor())

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Prepare the data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:02<00:00, 12908367.81it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209419.68it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3888946.92it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4566894.46it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
# Define the Autoencoder Neural Network
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3),  # Compressed representation
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid(),  # Using sigmoid to ensure the output is between 0 and 1
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the image
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = Autoencoder()

In [7]:
# Loss function
loss_fn = nn.MSELoss()

# Optimizer
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, _) in enumerate(dataloader):  # _ for targets, which we don't use
        # Compute prediction and loss
        reconstructed = model(X)
        loss = loss_fn(reconstructed, X.view(X.size(0), -1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
# Train the model
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.172198  [    0/60000]
loss: 0.059647  [ 6400/60000]
loss: 0.050731  [12800/60000]
loss: 0.041395  [19200/60000]
loss: 0.045119  [25600/60000]
loss: 0.043780  [32000/60000]
loss: 0.036790  [38400/60000]
loss: 0.037158  [44800/60000]
loss: 0.037371  [51200/60000]
loss: 0.035877  [57600/60000]
Epoch 2
-------------------------------
loss: 0.037145  [    0/60000]
loss: 0.034423  [ 6400/60000]
loss: 0.035728  [12800/60000]
loss: 0.033478  [19200/60000]
loss: 0.036684  [25600/60000]
loss: 0.035969  [32000/60000]
loss: 0.029795  [38400/60000]
loss: 0.032996  [44800/60000]
loss: 0.032961  [51200/60000]
loss: 0.030917  [57600/60000]
Epoch 3
-------------------------------
loss: 0.033837  [    0/60000]
loss: 0.031259  [ 6400/60000]
loss: 0.033152  [12800/60000]
loss: 0.032800  [19200/60000]
loss: 0.033760  [25600/60000]
loss: 0.034319  [32000/60000]
loss: 0.029007  [38400/60000]
loss: 0.031268  [44800/60000]
loss: 0.031831  [51200/60000]
loss: 0.02

In [10]:
# Test loop
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, _ in dataloader:
            reconstructed = model(X)
            test_loss += loss_fn(reconstructed, X.view(X.size(0), -1)).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

In [11]:
# Evaluate the model
test_loop(test_dataloader, model, loss_fn)

Avg loss: 0.028819 

